In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
train_data = datasets.ImageFolder(root="/kaggle/input/intel-image-classification/seg_train", transform=transform)
val_data = datasets.ImageFolder(root="/kaggle/input/intel-image-classification/seg_test", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [6]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 18 * 18, 512),  # Assuming input images are 150x150
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 6)  # 6 classes for classification
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc_layer(x)
        return x

In [7]:
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    return running_loss / len(train_loader), 100. * correct / total

In [9]:
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return running_loss / len(val_loader), 100. * correct / total

In [10]:
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.2f}%")

Epoch 1/10
Train Loss: 0.0040, Train Accuracy: 99.81%
Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 2/10
Train Loss: 0.0000, Train Accuracy: 100.00%
Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 3/10
Train Loss: 0.0000, Train Accuracy: 100.00%
Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 4/10
Train Loss: 0.0000, Train Accuracy: 100.00%
Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 5/10
Train Loss: 0.0000, Train Accuracy: 100.00%
Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 6/10
Train Loss: 0.0000, Train Accuracy: 100.00%
Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 7/10
Train Loss: 0.0000, Train Accuracy: 100.00%
Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 8/10
Train Loss: 0.0000, Train Accuracy: 100.00%
Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 9/10
Train Loss: 0.0000, Train Accuracy: 100.00%
Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 10/10
Train Loss: 0.0000, Train Accuracy: 100.00%
Val Loss: 0.0000, Val Accuracy: 100.00%


In [ ]:
torch.save(model, 'model.pth')